In [ ]:
import sys
from pathlib import Path

sys.path.insert(0, str(Path().resolve().parent.parent))

from src.server.tasks.os_interaction.task import Container

container = Container(image="local-os/default", sock_timeout=1)

In [ ]:
import socket
import struct
import re
from docker.transport.npipesocket import NpipeSocket
import time


def remove_ansi_codes(text):
    ansi_escape = re.compile(r"\x1B[@-_][0-?]*[ -/]*[@-~]")
    return ansi_escape.sub("", text)


def reset_socket(container: Container):
    kill_command = [
        "sh",
        "-c",
        "kill -9 $(ps aux | grep 'bash --login' | grep -v grep | awk '{print $2}')",
    ]
    exec_instance = container.client.api.exec_create(
        container.container.id, kill_command
    )
    exec_result = container.client.api.exec_start(exec_instance["Id"])

    container.exec_id = container.client.api.exec_create(
        container.container.id, "bash --login", stdin=True, tty=True
    )["Id"]
    container.sock = container.client.api.exec_start(container.exec_id, socket=True)
    container.sock.settimeout(5)
    # clear buffer
    container.sock.recv(1000)


class DummyOutput:
    output: str
    exit_code: int

    def __init__(self, code, o):
        self.output = o
        self.exit_code = code


def sock_read(sock: NpipeSocket, n: int, raise_err: bool = True):
    buffer = bytearray(n)
    try:
        sock.recv_into(buf=buffer, nbytes=n)
    except TimeoutError:
        if raise_err:
            raise
        else:
            return None
    return buffer


def read_data(sock: NpipeSocket):
    data = sock_read(sock, 8, raise_err=False)
    if not data:
        print("no data")
        return None
    _, n = struct.unpack(">BxxxL", data)
    return sock_read(sock, n)


def execute_short(sock: NpipeSocket, command: str):
    if not isinstance(command, str):
        return DummyOutput(-1, b"")

    # flush socket
    # sock_read(sock, 1000, raise_err=False)

    print("Send command:", command)
    command_bytes = command.encode("utf-8")
    sock.send(command_bytes + b"\n")
    pattern = b"\x1b.+@.+[#|$] "

    output = ""
    line = None
    while True:
        try:
            start = time.time()
            line = read_data(sock)
            if line is None:
                break
            line = re.sub(pattern, b'', line)
            s = remove_ansi_codes(line.decode("utf-8"))
            print("Line", (time.time() - start) * 1000, s)
            output += s
        except TimeoutError:
            break
        except socket.timeout:
            break
    cleaned_output = re.sub('\r', '', output)
    cleaned_output = re.sub(command, '', cleaned_output, count=1)
    print("---------RESULT---------:", cleaned_output)
    result = DummyOutput(0, cleaned_output)
    return result

In [ ]:
try:
    # command = 'cd ~'
    # command = 'cd /'
    # command = 'while true; do :; done'
    # command = 'cd /home'
    # command = 'ls'
    # command = 'pwd'
    command = """cd ~
    echo hello world > test.txt
    cat test.txt"""
    # command = 'cat test.txt'
    # command = """cat test.txt"""
    res = container.execute(command=command)
except Exception as ex:
    print(ex)
    reset_socket(container)

In [ ]:
res.output

In [ ]:
command

In [ ]:
cleaned = re.sub('\r', '', res.output)
cleaned

In [ ]:
re.sub(command, '', cleaned)

In [ ]:
command in cleaned

In [ ]:
list(command.encode())

zip_command_and_cleaned = list(zip(list(command.encode()), list(cleaned.encode())))
print(zip_command_and_cleaned)


In [ ]:
len(cleaned), len(command), len(zip_command_and_cleaned)

In [ ]:
bytes(list(map(lambda x: x[0], zip_command_and_cleaned[-4:]))).decode(), bytes(list(map(lambda x: x[1], zip_command_and_cleaned[-4:]))).decode()

In [ ]:
import itertools

# Using takewhile
result = list(
    map(
        lambda x: x[0],
        list(itertools.takewhile(lambda x: x[0] == x[1], zip_command_and_cleaned)),
    )
)

# Convert the result to a list and print it
print(result)

In [ ]:
bytes(result).decode()

In [ ]:
re.sub(command.encode(), b'', cleaned.encode())

In [ ]:
reset_socket(container)


In [ ]:
import win32file


def close_handle(cont: Container):
    time.sleep(2)
    print("closing handle")
    win32file.CancelIo(cont.sock._handle)


# t = Thread(target=close_handle, args=(container,))
# t.start()
data1 = sock_read(container.sock, 1000, raise_err=False)
print("data1", data1)
data2 = sock_read(container.sock, 1000, raise_err=False)
print("data2", data2)

In [ ]:
reset_socket(container)


In [ ]:
def read_always(container: Container, ev: Event):
    while True:
        while not ev.is_set():
            data = container.sock.recv(1000)
            print(data)

In [ ]:
kill_command = [
    "sh",
    "-c",
    "kill -9 $(ps aux | grep 'bash --login' | grep -v grep | awk '{print $2}')",
]
# kill_command =  ["sh", "-c", "ps aux | grep 'bash --login' | grep -v grep | awk '{print $2}'"]
exec_instance = container.client.api.exec_create(container.container.id, kill_command)
exec_result = container.client.api.exec_start(exec_instance["Id"])
exec_result